In [1]:
%load_ext sql
import sqlite3
conn = sqlite3.connect("ChicagoData.db")
cur = conn.cursor()
%sql sqlite:///ChicagoData.db


In [2]:
import pandas as pd
pd.read_csv('ChicagoCensusData.csv').to_sql('CENSUS_DATA', conn)
pd.read_csv('ChicagoPublicSchools.csv').to_sql('CHICAGO_PUBLIC_SCHOOLS', conn)
pd.read_csv('ChicagoCrimeData.csv').to_sql('CHICAGO_CRIME_DATA', conn)


533

# Joins
## Question 1
Write and execute a SQL query to list the school names, community names and average attendance for communities with a hardship index of 98.

In [9]:
%%sql
select school.NAME_OF_SCHOOL, school.COMMUNITY_AREA_NAME, school.AVERAGE_STUDENT_ATTENDANCE
from CHICAGO_PUBLIC_SCHOOLS as school left join CENSUS_DATA as census
on school.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER
where census.HARDSHIP_INDEX = 98


 * sqlite:///ChicagoData.db
Done.


NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,AVERAGE_STUDENT_ATTENDANCE
George Washington Carver Military Academy High School,RIVERDALE,91.60%
George Washington Carver Primary School,RIVERDALE,90.90%
Ira F Aldridge Elementary School,RIVERDALE,92.90%
William E B Dubois Elementary School,RIVERDALE,93.30%


## Question 2
Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [21]:
%%sql
select crime.CASE_NUMBER, crime.PRIMARY_TYPE, census.COMMUNITY_AREA_NAME
from CHICAGO_CRIME_DATA as crime left join CENSUS_DATA as census
on crime.COMMUNITY_AREA_NUMBER = census.COMMUNITY_AREA_NUMBER
where crime.LOCATION_DESCRIPTION like '%SCHOOL%'


 * sqlite:///ChicagoData.db
Done.


CASE_NUMBER,PRIMARY_TYPE,COMMUNITY_AREA_NAME
HL353697,BATTERY,South Shore
HL725506,BATTERY,Lincoln Square
HP716225,BATTERY,Douglas
HH639427,BATTERY,Austin
JA460432,BATTERY,Ashburn
HS200939,CRIMINAL DAMAGE,Austin
HK577020,NARCOTICS,Rogers Park
HS305355,NARCOTICS,Brighton Park
HT315369,ASSAULT,East Garfield Park
HR585012,CRIMINAL TRESPASS,Ashburn


# Creating A View
## Question 1
Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column.

Column name in CHICAGO_PUBLIC_SCHOOLS | Column name in view
---|---
NAME_OF_SCHOOL | School_Name
Safety_Icon | Safety_Rating
Family_Involvement_Icon | Family_Rating
Environment_Icon | Environment_Rating
Instruction_Icon | Instruction_Rating
Leaders_Icon | Leaders_Rating
Teachers_Icon | Teachers_Rating

Write and execute a SQL statement that returns all of the columns from the view.

Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [22]:
%%sql
create view school_info(School_Name, Safety_Rating, Family_Rating, Environment_Rating, Instruction_Rating, Leaders_Rating, Teachers_Rating) as
select NAME_OF_SCHOOL, Safety_Icon, Family_Involvement_Icon, Environment_Icon, Instruction_Icon, Leaders_Icon, Teachers_Icon
from CHICAGO_PUBLIC_SCHOOLS


 * sqlite:///ChicagoData.db
Done.


[]

In [24]:
%%sql
select * from school_info
limit 10


 * sqlite:///ChicagoData.db
Done.


School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating
Abraham Lincoln Elementary School,Very Strong,Very Strong,Strong,Strong,Weak,Strong
Adam Clayton Powell Paideia Community Academy Elementary School,Average,Strong,Strong,Very Strong,Weak,Strong
Adlai E Stevenson Elementary School,Strong,NDA,Average,Weak,Weak,NDA
Agustin Lara Elementary Academy,Average,Average,Average,Weak,Weak,Average
Air Force Academy High School,Average,Strong,Strong,Average,Weak,Average
Albany Park Multicultural Academy,Strong,Weak,Strong,Strong,Weak,Average
Albert G Lane Technical High School,Very Strong,NDA,Strong,Average,Weak,NDA
Albert R Sabin Elementary Magnet School,Strong,NDA,Weak,Very Weak,Weak,NDA
Alcott High School for the Humanities,Strong,NDA,Strong,Average,Weak,NDA
Alessandro Volta Elementary School,Average,Strong,Weak,Weak,Weak,Average


In [26]:
%%sql
select School_Name, Leaders_Rating
from school_info
limit 10


 * sqlite:///ChicagoData.db
Done.


School_Name,Leaders_Rating
Abraham Lincoln Elementary School,Weak
Adam Clayton Powell Paideia Community Academy Elementary School,Weak
Adlai E Stevenson Elementary School,Weak
Agustin Lara Elementary Academy,Weak
Air Force Academy High School,Weak
Albany Park Multicultural Academy,Weak
Albert G Lane Technical High School,Weak
Albert R Sabin Elementary Magnet School,Weak
Alcott High School for the Humanities,Weak
Alessandro Volta Elementary School,Weak


# Stored Procedures

## Question 1
Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer.

```sql
DELIMITER //
CREATE PROCEDURE UPDATE_LEADERS_SCORE(
    in in_School_ID integer,
    in in_Leader_Score integer
)
BEGIN

END //
DELIMITER ;
```

## Question 2
Inside your stored procedure, write a SQL statement to update the Leaders_Score field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID to the value in the in_Leader_Score parameter.


```sql
DELIMITER //
CREATE PROCEDURE UPDATE_LEADERS_SCORE(
    in in_School_ID integer,
    in in_Leader_Score integer
)
BEGIN

   UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Score = in_Leader_Score
    WHERE School_ID = in_School_ID;

END //
DELIMITER ;
```


## Question 3
Inside your stored procedure, write a SQL IF statement to update the Leaders_Icon field in the CHICAGO_PUBLIC_SCHOOLS table for the school identified by in_School_ID using the following information.

Score lower limit | Score upper limit | Icon
---|---|---
80 | 99 | Very strong
60 | 79 | Strong
40 | 59 | Average
20 | 39 | Weak
0 | 19 | Very weak

```sql
DELIMITER //
CREATE PROCEDURE UPDATE_LEADERS_SCORE(
    in in_School_ID integer,
    in in_Leader_Score integer
)
BEGIN

   UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Score = in_Leader_Score
    WHERE School_ID = in_School_ID;

    IF in_Leader_Score >= 80 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Very strong'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 60 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Strong'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 40 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Average'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 20 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Weak'
        WHERE School_ID = in_School_ID;
    ELSE
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Very weak'
        WHERE School_ID = in_School_ID;
    END IF;

END //
DELIMITER ;
```

## Question 4
Run your code to create the stored procedure.

```sql
%%sql
DELIMITER //
CREATE PROCEDURE UPDATE_LEADERS_SCORE(
    in in_School_ID integer,
    in in_Leader_Score integer
)
BEGIN

   UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Score = in_Leader_Score
    WHERE School_ID = in_School_ID;

    IF in_Leader_Score >= 80 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Very strong'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 60 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Strong'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 40 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Average'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 20 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Weak'
        WHERE School_ID = in_School_ID;
    ELSE
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Very weak'
        WHERE School_ID = in_School_ID;
    END IF;

END //
DELIMITER ;
```


# Using Transactions
## Question 1
Update your stored procedure definition. Add a generic ELSE clause to the IF statement that rolls back the current work if the score did not fit any of the preceding categories.

```sql
DELIMITER //
CREATE PROCEDURE UPDATE_LEADERS_SCORE(
    in in_School_ID integer,
    in in_Leader_Score integer
)
BEGIN
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        ROLLBACK;
        RESIGNAL;
    END;                
     
    START TRANSACTION;

    UPDATE CHICAGO_PUBLIC_SCHOOLS
    SET Leaders_Score = in_Leader_Score
    WHERE School_ID = in_School_ID;

    IF in_Leader_Score > 100 THEN
        ROLLBACK;
    ELSEIF in_Leader_Score >= 80 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Very strong'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 60 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Strong'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 40 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Average'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 20 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Weak'
        WHERE School_ID = in_School_ID;
    ELSEIF in_Leader_Score >= 0 THEN
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET Leaders_Icon = 'Very weak'
        WHERE School_ID = in_School_ID;
    ELSE
        ROLLBACK;
    END IF;

    COMMIT;

END //
DELIMITER ;
```


## Question 2
Update your stored procedure definition again. Add a statement to commit the current unit of work at the end of the procedure.